In [1]:
import os.path as osp
import openslide
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
# 이부분 python 에서는 뺴주기

from skimage.filters import threshold_otsu
from openslide.deepzoom import DeepZoomGenerator
import cv2
from keras.utils.np_utils import to_categorical

# network
from keras.models import Sequential
from keras.layers import Lambda, Dropout
from keras.layers.convolutional import Convolution2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.models import load_model

# Unet
import numpy as np 
import os

import skimage.transform as trans
#import numpy as np
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras


# train

from sklearn.model_selection import StratifiedShuffleSplit
from datetime import datetime

# evaluate
import matplotlib.gridspec as gridspec
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

import math
from PIL import Image
from xml.etree.ElementTree import ElementTree, Element, SubElement
from io import BytesIO
import skimage.io as io

from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())
import keras.backend.tensorflow_backend as K
from sklearn import metrics

from keras.preprocessing.image import *

Using TensorFlow backend.


In [2]:
PATCH_SIZE = 256
IS_TRAIN = True
def find_patches_from_slide(slide_path, truth_path, patch_size=PATCH_SIZE,filter_non_tissue=True,filter_only_all_tumor=True):
    
    slide_contains_tumor = 'pos' in slide_path
    
    ############### read_region을 위한 start, level, size를 구함 #######################
    BOUNDS_OFFSET_PROPS = (openslide.PROPERTY_NAME_BOUNDS_X, openslide.PROPERTY_NAME_BOUNDS_Y)
    BOUNDS_SIZE_PROPS = (openslide.PROPERTY_NAME_BOUNDS_WIDTH, openslide.PROPERTY_NAME_BOUNDS_HEIGHT)

    if slide_contains_tumor:
        with openslide.open_slide(slide_path) as slide:
            start = (int(slide.properties.get('openslide.bounds-x',0)),int(slide.properties.get('openslide.bounds-y',0)))
            level = np.log2(patch_size) 
            level = int(level)
            
            size_scale = tuple(int(slide.properties.get(prop, l0_lim)) / l0_lim
                            for prop, l0_lim in zip(BOUNDS_SIZE_PROPS,
                            slide.dimensions))
            _l_dimensions = tuple(tuple(int(math.ceil(l_lim * scale))
                            for l_lim, scale in zip(l_size, size_scale))
                            for l_size in slide.level_dimensions)
            size = _l_dimensions[level]
            
            slide4 = slide.read_region(start,level,size) 
    else :
        with openslide.open_slide(slide_path) as slide:
            start = (0,0)
            level = np.log2(patch_size) 
            level = int(level)
            
            size_scale = (1,1)
            _l_dimensions = tuple(tuple(int(math.ceil(l_lim * scale))
                            for l_lim, scale in zip(l_size, size_scale))
                            for l_size in slide.level_dimensions)
            size = _l_dimensions[level]
            
            slide4 = slide.read_region(start,level,size) 
    ####################################################################################
    
    
    # is_tissue 부분 
    slide4_grey = np.array(slide4.convert('L'))
    binary = slide4_grey > 0  # black이면 0임
    
    # 검은색 제외하고 흰색영역(배경이라고 여겨지는)에 대해서도 작업해주어야함.
    slide4_not_black = slide4_grey[slide4_grey>0]
    thresh = threshold_otsu(slide4_not_black)
    
    I, J = slide4_grey.shape
    for i in range(I):
        for j in range(J):
            if slide4_grey[i,j] > thresh :
                binary[i,j] = False
    patches = pd.DataFrame(pd.DataFrame(binary).stack())
    patches['is_tissue'] = patches[0]
    patches.drop(0, axis=1,inplace =True)
    patches.loc[:,'slide_path'] = slide_path
    
#     # Test 이면 
#     if IS_TEST:
#         return patches

    # is_tumor 부분
    if slide_contains_tumor:
        with openslide.open_slide(truth_path) as truth:
            thumbnail_truth = truth.get_thumbnail(size) 
        
        patches_y = pd.DataFrame(pd.DataFrame(np.array(thumbnail_truth.convert("L"))).stack())
        patches_y['is_tumor'] = patches_y[0] > 0
        
        # mask된 영역이 애매할 수도 있으므로
        patches_y['is_all_tumor'] = patches_y[0] == 255
        patches_y.drop(0, axis=1, inplace=True)
        samples = pd.concat([patches, patches_y], axis=1) #len(samples)
    else:
        samples = patches
        #dfmi.loc[:,('one','second')] = value
        samples.loc[:,'is_tumor'] = False
        samples.loc[:,'is_all_tumor'] = False
    
    if filter_non_tissue:
        samples = samples[samples.is_tissue == True] # remove patches with no tissue #samples = samples[samples.is_tissue == True]
    
    if filter_only_all_tumor :
        samples['tile_loc'] = list(samples.index)
        all_tissue_samples1 = samples[samples.is_tumor==False]
        all_tissue_samples1 = all_tissue_samples1.append(samples[samples.is_all_tumor==True])
        
        all_tissue_samples1.reset_index(inplace=True, drop=True)
    else :
        return samples
    
    return all_tissue_samples1
NUM_CLASSES = 2 # not_tumor, tumor

file_handles=[]
def gen_imgs(all_image_path, all_mask_path, samples, batch_size, patch_size = PATCH_SIZE, shuffle=True):
   
    num_samples = len(samples)
    # 특정 몇개의 slide만 open 해서 쓰기
    # 4개씩 묶었으니까 
  
    slide_path0 = all_image_path[0]
    slide_path1 = all_image_path[1]
    slide_path2 = all_image_path[2]
    slide_path3 = all_image_path[3]
    
    
    # slide 0~3 까지 미리 열어두기
    slide0 = openslide.open_slide(slide_path0)
    slide1 = openslide.open_slide(slide_path1)
    slide2 = openslide.open_slide(slide_path2)
    slide3 = openslide.open_slide(slide_path3)
    file_handles.append(slide0)
    file_handles.append(slide1)
    file_handles.append(slide2)
    file_handles.append(slide3)
    
    # with openslide.open_slide(slide_path) as slide
    tiles0 = DeepZoomGenerator(slide0,tile_size=patch_size, overlap=0, limit_bounds=False) 
    tiles1 = DeepZoomGenerator(slide1,tile_size=patch_size, overlap=0, limit_bounds=False)
    tiles2 = DeepZoomGenerator(slide2,tile_size=patch_size, overlap=0, limit_bounds=False)
    tiles3 = DeepZoomGenerator(slide3,tile_size=patch_size, overlap=0, limit_bounds=False)
    
    
    if 'pos' in slide_path0:
        start_x0 = int(slide0.properties.get('openslide.bounds-x',0))
        start_y0 = int(slide0.properties.get('openslide.bounds-y',0))
        start_x0 = start_x0 / patch_size
        start_y0 = start_y0 / patch_size
        
        truth0 = openslide.open_slide(all_mask_path[0])
        truth_tiles0 = DeepZoomGenerator(truth0, tile_size=16, overlap=0, limit_bounds=False) 
        
    else : 
        start_x0 = 0
        start_y0 = 0
    
    if 'pos' in slide_path1:
        start_x1 = int(slide1.properties.get('openslide.bounds-x',0))
        start_y1 = int(slide1.properties.get('openslide.bounds-y',0))
        start_x1 = start_x1 / patch_size
        start_y1 = start_y1 / patch_size
        
        truth1 = openslide.open_slide(all_mask_path[1])
        truth_tiles1 = DeepZoomGenerator(truth1, tile_size=16, overlap=0, limit_bounds=False) 
        
    else : 
        start_x1 = 0
        start_y1 = 0
    
    if 'pos' in slide_path2:
        start_x2 = int(slide2.properties.get('openslide.bounds-x',0))
        start_y2 = int(slide2.properties.get('openslide.bounds-y',0))
        start_x2 = start_x2 / patch_size
        start_y2 = start_y2 / patch_size
        
        truth2 = openslide.open_slide(all_mask_path[2])
        truth_tiles2 = DeepZoomGenerator(truth2, tile_size=16, overlap=0, limit_bounds=False) 
        
    else : 
        start_x2 = 0
        start_y2 = 0
        
    if 'pos' in slide_path3:
        start_x3 = int(slide3.properties.get('openslide.bounds-x',0))
        start_y3 = int(slide3.properties.get('openslide.bounds-y',0))
        start_x3 = start_x3 / patch_size
        start_y3 = start_y3 / patch_size
        
        truth3 = openslide.open_slide(all_mask_path[3])
        truth_tiles3 = DeepZoomGenerator(truth3, tile_size=16, overlap=0, limit_bounds=False) 
        
    else : 
        start_x3 = 0
        start_y3 = 0
    

    
    for epo in range(10): # Loop forever so the generator never terminates
        if shuffle:
            samples = samples.sample(frac=1) # shuffle samples

        for offset in range(0, num_samples, batch_size):
            batch_samples = samples.iloc[offset:offset+batch_size]
            images = []
            masks = []
            for _, batch_sample in batch_samples.iterrows(): # 배치마다 deep zoom 하네 약간 비효율적
                
                # 여기서 하나씩 4개 체크해서 해당되는 부분으로 가야지. for 4번 돌리면서 가야한다.
                mask_size_up = np.zeros((patch_size,patch_size))
                a,b=mask_size_up.shape
                
                if batch_sample.slide_path == slide_path0:
                    x, y = batch_sample.tile_loc[::-1]
                    x += start_x0
                    y += start_y0
                    img = tiles0.get_tile(tiles0.level_count-1, (x,y))
                    if 'pos' in slide_path0:
                        mask = truth_tiles0.get_tile(truth_tiles0.level_count-1, batch_sample.tile_loc[::-1])
                        mask = (cv2.cvtColor(np.array(mask), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                            # mask_size_up , 16 to 256
                        for i in range(a):
                            for j in range(b) :
                                k = i//16
                                l = j//16
                                mask_size_up[i,j] = mask[k,l]
                    
                elif batch_sample.slide_path == slide_path1:
                    x, y = batch_sample.tile_loc[::-1]
                    x += start_x1
                    y += start_y1
                    img = tiles1.get_tile(tiles1.level_count-1, (x,y))
                    if 'pos' in slide_path1:
                        mask = truth_tiles1.get_tile(truth_tiles1.level_count-1, batch_sample.tile_loc[::-1])
                        mask = (cv2.cvtColor(np.array(mask), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                            # mask_size_up , 16 to 256
                        for i in range(a):
                            for j in range(b) :
                                k = i//16
                                l = j//16
                                mask_size_up[i,j] = mask[k,l]
                
                elif batch_sample.slide_path == slide_path2:
                    x, y = batch_sample.tile_loc[::-1]
                    x += start_x2
                    y += start_y2
                    img = tiles2.get_tile(tiles2.level_count-1, (x,y))
                    if 'pos' in slide_path2:
                        mask = truth_tiles2.get_tile(truth_tiles2.level_count-1, batch_sample.tile_loc[::-1])
                        mask = (cv2.cvtColor(np.array(mask), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                            # mask_size_up , 16 to 256
                        for i in range(a):
                            for j in range(b) :
                                k = i//16
                                l = j//16
                                mask_size_up[i,j] = mask[k,l]
                
                else:
                    x, y = batch_sample.tile_loc[::-1]
                    x += start_x3
                    y += start_y3
                    img = tiles3.get_tile(tiles3.level_count-1, (x,y))
                    if 'pos' in slide_path3:
                        mask = truth_tiles3.get_tile(truth_tiles3.level_count-1, batch_sample.tile_loc[::-1])
                        mask = (cv2.cvtColor(np.array(mask), cv2.COLOR_RGB2GRAY) > 0).astype(int)
                            # mask_size_up , 16 to 256
                        for i in range(a):
                            for j in range(b) :
                                k = i//16
                                l = j//16
                                mask_size_up[i,j] = mask[k,l]
                
                    

                images.append(np.array(img))
                masks.append(mask_size_up)

            X_train = np.array(images)
            y_train = np.array(masks)
            #print('x_train_shape :', X_train.shape)
            
            y_train = to_categorical(y_train, num_classes=2).reshape(y_train.shape[0], patch_size, patch_size, 2) 
            #print('y_train_shape : ',y_train.shape)
            
            #X_train, y_train = datagen().flow(X_train,y = y_train,batch_size = batch_size)
            X_train, y_train = next(ImageDataGenerator(
                rotation_range=90,
                horizontal_flip=True,
                vertical_flip=True,
                brightness_range =(0.25,1.)).flow(X_train,y=y_train,batch_size=batch_size))
            #print(X_train.shape)
            #print(y_train.shape)
            yield X_train, y_train
            
def predict_batch_from_model(patches, model):
    """Predict which pixels are tumor.
    
    input: patch: `batch_size`x256x256x3, rgb image
    input: model: keras model
    output: prediction: 256x256x1, per-pixel tumor probability
    """
    predictions = model.predict(patches)
    predictions = predictions[:, :, :, 1]
    return predictions
def predict_from_model(patch, model):
    """Predict which pixels are tumor.
    
    input: patch: 256x256x3, rgb image
    input: model: keras model
    output: prediction: 256x256x1, per-pixel tumor probability
    """
    
    prediction = model.predict(patch.reshape(1, 256, 256, 3))
    prediction = prediction[:, :, :, 1].reshape(256, 256)
    return prediction

def predict_from_model_n(patch, model):
    """Predict which pixels are tumor.
    
    input: patch: 256x256x3, rgb image
    input: model: keras model
    output: prediction: 256x256x1, per-pixel tumor probability
    """
    
    prediction = model.predict(patch.reshape(1, 256, 256, 3))
    prediction = prediction[:, :, :, 0].reshape(256, 256)
    return prediction

def simple_model(pretrained_weights = None):
    model = Sequential()
    model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(256, 256, 3)))
    model.add(Convolution2D(100, (3, 3), strides=(2, 2), activation='elu', padding='same'))
    model.add(MaxPooling2D())
    model.add(Convolution2D(200, (3, 3), strides=(2, 2), activation='elu', padding='same'))
    model.add(MaxPooling2D())
    model.add(Convolution2D(300, (3, 3), activation='elu', padding='same'))
    model.add(Convolution2D(300, (3, 3), activation='elu',  padding='same'))
    model.add(Dropout(0.2))
    model.add(Convolution2D(2, (1, 1))) # this is called upscore layer for some reason?
    model.add(Conv2DTranspose(2, (31, 31), strides=(16, 16), activation='softmax', padding='same'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
        
    return model

In [3]:
def read_data_path():
    image_paths = []
    with open('train.txt','r') as f:
        for line in f:
            line = line.rstrip('\n')
            image_paths.append(line)
    #print('image_path # : ',len(image_paths))

    tumor_mask_paths = []

    with open('train_mask.txt','r') as f:
        for line in f:
            line = line.rstrip('\n')
            tumor_mask_paths.append(line)
    #print('mask_patch # : ',len(tumor_mask_paths))
    
    return image_paths, tumor_mask_paths

def read_test_data_path():
    image_paths = []
    with open('test.txt','r') as f:
        for line in f:
            line = line.rstrip('\n')
            image_paths.append(line)
    #print('image_path # : ',len(image_paths))
    
    return image_paths

test_image_paths = read_test_data_path()
image_paths, tumor_mask_paths = read_data_path()
image_paths = []
with open('train.txt','r') as f:
    for line in f:
        line = line.rstrip('\n')
        image_paths.append(line)
print('image_path # : ',len(image_paths))

tumor_mask_paths = []
with open('train_mask.txt','r') as f:
    for line in f:
        line = line.rstrip('\n')
        tumor_mask_paths.append(line)
print('mask_patch # : ',len(tumor_mask_paths))

slide_4_list_1 = [[102,104,29,44],[144,55,30,18],[125,56,35,40],[54,65,21,36],[139,82,1,49],[73,108,7,23],[107,117,24,52],[106,103,27,13]
               ,[105,151,15,2],[75,100,41,9],[156,113,32,37],[150,88,39,10],[84,122,5,50],[93,118,53,47],[87,78,45,34],[116,98,48,46],
                [72,131,22,42]]
slide_4_list_2 = [[109,58,14,28],[101,69,11,43],[94,74,3,20],[64,140,17,16],[92,154,8,26],[99,60,0,33],[86,146,25,19],[68,112,38,51],
                 [71,136,31,4],[59,91,12,6]]
slide_4_list_3 = [[143,132,124,85],[95,120,81,77],[97,96,110,83],[152,128,149,155],[153,111,57,138],[134,135,114,76],
                  [123,90,121,61],[147,148,119,142],[66,137,63,80],[70,79,115,133],[129,141,127,145]]
slide_4_test = [[55,55,0,0],[55,55,0,0]]

all_image_path = []
all_mask_path = []
for j in range(4):
    image_path = image_paths[slide_4_test[0][j]][1:] # 이 부분은 data 읽을때 고치자 ( [1:] 빼야함)
    mask_path = tumor_mask_paths[slide_4_test[0][j]][1:] # 이 부분은 data 읽을때 고치자
    all_image_path.append(image_path)
    all_mask_path.append(mask_path)

image_path # :  157
mask_patch # :  157


In [6]:
def unet(pretrained_weights = None,input_size = (256,256,3)):
    inputs = Input(input_size)
    inputs_norm = Lambda(lambda x: x /255.0 - 0.5)(inputs)
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs_norm)
    conv1 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(16, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(2, 1, activation = 'softmax')(conv9)

    model = Model(input = inputs, output = conv10)

    #model.compile(optimizer = Adam(lr = 1e-2), loss = 'binary_crossentropy', metrics = ['accuracy'])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    #model.summary()

    if(pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [8]:
model = unet('unet_1.h5')

d:\anaconda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:44: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


In [10]:
columns = ['is_tissue','slide_path','is_tumor','is_all_tumor','tile_loc']

BATCH_SIZE = 64
N_EPOCHS = 2

for i in range(len(slide_4_test)):
    
    # [1] dataset , 2 pos, 2 neg, mean ratio = 3:1
    four_samples = pd.DataFrame(columns = columns)
    four_image_path = list()
    four_mask_path = list()    
    for j in range(4):
        image_path = image_paths[slide_4_test[i][j]][1:] # 이 부분은 data 읽을때 고치자 ( [1:] 빼야함)
        mask_path = tumor_mask_paths[slide_4_test[i][j]][1:] # 이 부분은 data 읽을때 고치자
        samples = find_patches_from_slide(image_path, mask_path)
        
        four_samples = four_samples.append(samples)   
        four_image_path.append(image_path)
        four_mask_path.append(mask_path)
    NUM_SAMPLES = len(four_samples)
    if NUM_SAMPLES > 50000:
        NUM_SAMPLES = 50000
    
    samples = four_samples.sample(NUM_SAMPLES, random_state=42)
    samples.reset_index(drop=True, inplace=True)
    
    tumor_samples = four_samples[four_samples.is_tumor == True]
    print(len(tumor_samples))
    non_tumor_samples = four_samples[four_samples.is_tumor == False]
    print(len(non_tumor_samples))
    non_tumor_samples_3_ratio = non_tumor_samples.sample(len(tumor_samples) * 3, random_state = 42,replace=True)
    
    all_sample = tumor_samples.append(non_tumor_samples_3_ratio)

    split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
    for train_index, test_index in split.split(samples, samples["is_tumor"]):
            train_samples = samples.loc[train_index]
            validation_samples = samples.loc[test_index]
    
    train_generator = gen_imgs(four_image_path,four_mask_path,train_samples, BATCH_SIZE)
    validation_generator = gen_imgs(four_image_path,four_mask_path,validation_samples, BATCH_SIZE)
    
    train_start_time = datetime.now()
    history = model.fit_generator(train_generator, np.ceil(len(train_samples) / BATCH_SIZE),
        validation_data=validation_generator,
        validation_steps=np.ceil(len(validation_samples) / BATCH_SIZE),
        epochs=N_EPOCHS)
    if file_handles != []:
        for fh in file_handles:
            fh.close()
    file_handles=[]
    #del train_generator
    #del validation_generator
    train_end_time = datetime.now()
    print("Model training time: %.1f minutes" % ((train_end_time - train_start_time).seconds / 60,))
    model.save('unet_1.h5')
    # split
    # data gen : all_image_path, all_mask_path

d:\anaconda\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


61446
160136
Epoch 1/2


204/704 [=======>......................] - ETA: 1:34:28 - loss: 0.1088 - acc: 0.96 - ETA: 58:30 - loss: 0.1176 - acc: 0.9651 - ETA: 56:46 - loss: 0.1220 - acc: 0.96 - ETA: 55:22 - loss: 0.1288 - acc: 0.95 - ETA: 54:43 - loss: 0.1557 - acc: 0.95 - ETA: 54:05 - loss: 0.1658 - acc: 0.94 - ETA: 53:32 - loss: 0.1753 - acc: 0.94 - ETA: 53:15 - loss: 0.1661 - acc: 0.94 - ETA: 53:09 - loss: 0.1610 - acc: 0.95 - ETA: 52:53 - loss: 0.1531 - acc: 0.95 - ETA: 52:41 - loss: 0.1482 - acc: 0.95 - ETA: 51:12 - loss: 0.1563 - acc: 0.94 - ETA: 49:39 - loss: 0.1526 - acc: 0.95 - ETA: 48:21 - loss: 0.1519 - acc: 0.95 - ETA: 47:14 - loss: 0.1475 - acc: 0.95 - ETA: 46:14 - loss: 0.1476 - acc: 0.95 - ETA: 45:22 - loss: 0.1465 - acc: 0.95 - ETA: 44:36 - loss: 0.1441 - acc: 0.95 - ETA: 43:53 - loss: 0.1392 - acc: 0.95 - ETA: 43:15 - loss: 0.1363 - acc: 0.95 - ETA: 42:39 - loss: 0.1381 - acc: 0.95 - ETA: 42:07 - loss: 0.1376 - acc: 0.95 - ETA: 41:39 - loss: 0.1355 - acc: 0.95 - ETA: 41:10 - loss: 0.1364 - acc: 

408/704 [================>.............] - ETA: 24:37 - loss: 0.0976 - acc: 0.96 - ETA: 24:34 - loss: 0.0978 - acc: 0.96 - ETA: 24:30 - loss: 0.0978 - acc: 0.96 - ETA: 24:27 - loss: 0.0979 - acc: 0.96 - ETA: 24:24 - loss: 0.0977 - acc: 0.96 - ETA: 24:21 - loss: 0.0977 - acc: 0.96 - ETA: 24:17 - loss: 0.0976 - acc: 0.96 - ETA: 24:14 - loss: 0.0978 - acc: 0.96 - ETA: 24:11 - loss: 0.0980 - acc: 0.96 - ETA: 24:08 - loss: 0.0978 - acc: 0.96 - ETA: 24:05 - loss: 0.0974 - acc: 0.96 - ETA: 24:02 - loss: 0.0974 - acc: 0.96 - ETA: 23:58 - loss: 0.0974 - acc: 0.96 - ETA: 23:55 - loss: 0.0975 - acc: 0.96 - ETA: 23:52 - loss: 0.0975 - acc: 0.96 - ETA: 23:49 - loss: 0.0975 - acc: 0.96 - ETA: 23:46 - loss: 0.0975 - acc: 0.96 - ETA: 23:43 - loss: 0.0972 - acc: 0.96 - ETA: 23:40 - loss: 0.0972 - acc: 0.96 - ETA: 23:37 - loss: 0.0972 - acc: 0.96 - ETA: 23:34 - loss: 0.0971 - acc: 0.96 - ETA: 23:31 - loss: 0.0970 - acc: 0.96 - ETA: 23:27 - loss: 0.0971 - acc: 0.96 - ETA: 23:24 - loss: 0.0968 - acc: 0.96

612/704 [=========================>....] - ETA: 14:18 - loss: 0.0954 - acc: 0.96 - ETA: 14:15 - loss: 0.0954 - acc: 0.96 - ETA: 14:12 - loss: 0.0955 - acc: 0.96 - ETA: 14:09 - loss: 0.0960 - acc: 0.96 - ETA: 14:07 - loss: 0.0961 - acc: 0.96 - ETA: 14:04 - loss: 0.0961 - acc: 0.96 - ETA: 14:01 - loss: 0.0961 - acc: 0.96 - ETA: 13:58 - loss: 0.0961 - acc: 0.96 - ETA: 13:55 - loss: 0.0962 - acc: 0.96 - ETA: 13:52 - loss: 0.0961 - acc: 0.96 - ETA: 13:49 - loss: 0.0961 - acc: 0.96 - ETA: 13:46 - loss: 0.0961 - acc: 0.96 - ETA: 13:43 - loss: 0.0960 - acc: 0.96 - ETA: 13:40 - loss: 0.0960 - acc: 0.96 - ETA: 13:37 - loss: 0.0960 - acc: 0.96 - ETA: 13:35 - loss: 0.0964 - acc: 0.96 - ETA: 13:32 - loss: 0.0964 - acc: 0.96 - ETA: 13:29 - loss: 0.0964 - acc: 0.96 - ETA: 13:26 - loss: 0.0963 - acc: 0.96 - ETA: 13:23 - loss: 0.0965 - acc: 0.96 - ETA: 13:20 - loss: 0.0965 - acc: 0.96 - ETA: 13:17 - loss: 0.0965 - acc: 0.96 - ETA: 13:14 - loss: 0.0965 - acc: 0.96 - ETA: 13:11 - loss: 0.0964 - acc: 0.96

704/704 [==============================] - ETA: 4:24 - loss: 0.0954 - acc: 0.966 - ETA: 4:21 - loss: 0.0954 - acc: 0.966 - ETA: 4:18 - loss: 0.0955 - acc: 0.966 - ETA: 4:15 - loss: 0.0955 - acc: 0.966 - ETA: 4:12 - loss: 0.0954 - acc: 0.966 - ETA: 4:09 - loss: 0.0955 - acc: 0.966 - ETA: 4:06 - loss: 0.0955 - acc: 0.966 - ETA: 4:03 - loss: 0.0954 - acc: 0.966 - ETA: 4:00 - loss: 0.0954 - acc: 0.966 - ETA: 3:58 - loss: 0.0953 - acc: 0.966 - ETA: 3:55 - loss: 0.0953 - acc: 0.966 - ETA: 3:52 - loss: 0.0953 - acc: 0.966 - ETA: 3:49 - loss: 0.0953 - acc: 0.966 - ETA: 3:46 - loss: 0.0954 - acc: 0.966 - ETA: 3:43 - loss: 0.0954 - acc: 0.966 - ETA: 3:40 - loss: 0.0954 - acc: 0.966 - ETA: 3:37 - loss: 0.0956 - acc: 0.965 - ETA: 3:34 - loss: 0.0956 - acc: 0.965 - ETA: 3:31 - loss: 0.0957 - acc: 0.965 - ETA: 3:28 - loss: 0.0957 - acc: 0.965 - ETA: 3:26 - loss: 0.0957 - acc: 0.965 - ETA: 3:23 - loss: 0.0957 - acc: 0.965 - ETA: 3:20 - loss: 0.0957 - acc: 0.965 - ETA: 3:17 - loss: 0.0956 - acc: 0.965

200/704 [=======>......................] - ETA: 5:08 - loss: 0.1100 - acc: 0.969 - ETA: 5:26 - loss: 0.0808 - acc: 0.974 - ETA: 5:37 - loss: 0.0912 - acc: 0.969 - ETA: 5:53 - loss: 0.1013 - acc: 0.962 - ETA: 5:55 - loss: 0.1126 - acc: 0.957 - ETA: 5:58 - loss: 0.1122 - acc: 0.956 - ETA: 6:02 - loss: 0.1093 - acc: 0.957 - ETA: 5:53 - loss: 0.1062 - acc: 0.959 - ETA: 5:47 - loss: 0.1075 - acc: 0.960 - ETA: 5:41 - loss: 0.1168 - acc: 0.957 - ETA: 5:45 - loss: 0.1115 - acc: 0.959 - ETA: 5:42 - loss: 0.1113 - acc: 0.958 - ETA: 8:40 - loss: 0.1070 - acc: 0.960 - ETA: 11:36 - loss: 0.1010 - acc: 0.96 - ETA: 14:12 - loss: 0.1034 - acc: 0.96 - ETA: 16:31 - loss: 0.1174 - acc: 0.95 - ETA: 18:32 - loss: 0.1168 - acc: 0.96 - ETA: 20:22 - loss: 0.1125 - acc: 0.96 - ETA: 22:00 - loss: 0.1137 - acc: 0.96 - ETA: 23:24 - loss: 0.1116 - acc: 0.96 - ETA: 24:41 - loss: 0.1142 - acc: 0.96 - ETA: 25:45 - loss: 0.1130 - acc: 0.96 - ETA: 26:01 - loss: 0.1152 - acc: 0.95 - ETA: 26:14 - loss: 0.1126 - acc: 0.96

KeyboardInterrupt: 

In [11]:
model.save('unet_1.h5')

# Test

In [12]:
ipath = all_image_path[0]
tpath = all_mask_path[0]

all_tissue_samples = find_patches_from_slide(ipath,tpath)
print('Total patches in slide: %d' % len(all_tissue_samples)) 
all_tissue_samples.iloc[:10]
all_tissue_samples.is_tumor.value_counts() 
test_start_time = datetime.now()
sample_gen = gen_imgs(all_image_path,all_mask_path,all_tissue_samples, 5000, shuffle=True)
%time example_X, example_y  = next(sample_gen)
test_end_time = datetime.now()
print("5000 gen time: %.1f minutes" % ((test_end_time - test_start_time).seconds / 60,))

start_x = PATCH_SIZE//4
start_y = PATCH_SIZE//4
pred_size = PATCH_SIZE//2

test_start_time = datetime.now()
preds = []
labels = []
for i in range(5000):
    prediction = predict_from_model(example_X[i],model)
    pred_X = np.zeros((pred_size,pred_size))
    y = example_y[i].argmax
    for x in range(start_x,start_x+pred_size):
        for y in range(start_y, start_y+pred_size):
            pred_X[x-start_x][y-start_y] = prediction[x][y]
            
    pred_s = pd.Series(pred_X.flatten())
    max_p = np.max(pred_s)
    
    y = np.max(example_y[i].argmax(axis=2))
    preds.append(max_p)
    labels.append(y)
test_end_time = datetime.now()
print("Model test time: %.1f minutes" % ((test_end_time - test_start_time).seconds / 60,))    
fpr, tpr, thresholds = metrics.roc_curve(labels,preds,pos_label=1)
print(metrics.auc(fpr,tpr))

Total patches in slide: 105213
Wall time: 4min 25s
5000 gen time: 4.4 minutes
Model test time: 2.0 minutes
0.9951332079414998


In [13]:
ipath = all_image_path[0]
tpath = all_mask_path[0]

all_tissue_samples = find_patches_from_slide(ipath,tpath)
print('Total patches in slide: %d' % len(all_tissue_samples)) 
all_tissue_samples.iloc[:10]
all_tissue_samples.is_tumor.value_counts() 
test_start_time = datetime.now()
sample_gen = gen_imgs(all_image_path,all_mask_path,all_tissue_samples, 5000, shuffle=True)
%time example_X, example_y  = next(sample_gen)
test_end_time = datetime.now()
print("5000 gen time: %.1f minutes" % ((test_end_time - test_start_time).seconds / 60,))

start_x = PATCH_SIZE//4
start_y = PATCH_SIZE//4
pred_size = PATCH_SIZE//2

test_start_time = datetime.now()
preds = []
labels = []
for i in range(5000):
    prediction = predict_from_model(example_X[i],model)
    pred_X = np.zeros((pred_size,pred_size))
    y = example_y[i].argmax
    pred_s = pd.Series(prediction.flatten())
    max_p = np.max(pred_s)
    
    y = np.max(example_y[i].argmax(axis=2))
    preds.append(max_p)
    labels.append(y)
test_end_time = datetime.now()
print("Model test time: %.1f minutes" % ((test_end_time - test_start_time).seconds / 60,))    
fpr, tpr, thresholds = metrics.roc_curve(labels,preds,pos_label=1)
print(metrics.auc(fpr,tpr))

Total patches in slide: 105213
Wall time: 4min 33s
5000 gen time: 4.6 minutes
Model test time: 0.8 minutes
0.9957573195312768


In [14]:
ipath = all_image_path[0]
tpath = all_mask_path[0]

all_tissue_samples = find_patches_from_slide(ipath,tpath)
print('Total patches in slide: %d' % len(all_tissue_samples)) 
all_tissue_samples.iloc[:10]
all_tissue_samples.is_tumor.value_counts() 
test_start_time = datetime.now()
sample_gen = gen_imgs(all_image_path,all_mask_path,all_tissue_samples, 5000, shuffle=True)
%time example_X, example_y  = next(sample_gen)
test_end_time = datetime.now()
print("5000 gen time: %.1f minutes" % ((test_end_time - test_start_time).seconds / 60,))

start_x = 32
start_y = 32
pred_size = 192

test_start_time = datetime.now()
preds = []
labels = []
for i in range(5000):
    prediction = predict_from_model(example_X[i],model)
    pred_X = np.zeros((pred_size,pred_size))
    y = example_y[i].argmax
    for x in range(start_x,start_x+pred_size):
        for y in range(start_y, start_y+pred_size):
            pred_X[x-start_x][y-start_y] = prediction[x][y]
            
    pred_s = pd.Series(pred_X.flatten())
    max_p = np.max(pred_s)
    
    y = np.max(example_y[i].argmax(axis=2))
    preds.append(max_p)
    labels.append(y)
test_end_time = datetime.now()
print("Model test time: %.1f minutes" % ((test_end_time - test_start_time).seconds / 60,))    
fpr, tpr, thresholds = metrics.roc_curve(labels,preds,pos_label=1)
print(metrics.auc(fpr,tpr))

Total patches in slide: 105213
Wall time: 4min 37s
5000 gen time: 4.7 minutes
Model test time: 3.7 minutes
0.9960936531019771


In [17]:
max_pred_x = np.max(preds)
max_pred_x

min_pred_x = np.min(preds)
min_pred_x

0.0004315432452131063

In [23]:
test_start_time = datetime.now()
slide = openslide.open_slide(ipath)
tiles = DeepZoomGenerator(slide,tile_size=256,overlap=0, limit_bounds=False) 
test_end_time = datetime.now()
print("Model test time: %.1f minutes" % ((test_end_time - test_start_time).seconds))

Model test time: 0.0 minutes
